In [11]:
from __future__ import print_function
from __future__ import division
import tensorflow as tf
import scipy.io as sio
import numpy as np
from datetime import datetime
from pprint import pprint

In [12]:
from sklearn.cross_validation import train_test_split

In [13]:
A =  sio.loadmat('./HW2/A2P2.mat')['A']
X = A[:,0:-1]
Y = A[:,-1].reshape(10000,1)
trainX,testx,trainY,testy = train_test_split(X,Y,test_size = 0.1)

print(trainX.shape)
# Set model parameters
num_features = trainX.shape[1] # num of input nodes
num_labels = 1 # num of outcomes
pprint(num_features)
num_samples = trainX.shape[0]
h_size = 5 # num hidden layers
# Training parameters
training_epochs = 10000
n_hidden_1 = 100
n_hidden_2 = 80
n_hidden_3 = 50
n_hidden_4 = 10
n_hidden_5 = 1

epoch_size = 100
display_step = 500
learning_rate = 0.01

# Set the directory to specify where the summary will be written
summaries_dir = "./HW2/SummaryDir" + datetime.now().strftime('%Y%m%d_%H_%M_%S')

# pprint(trainX[:,1])

(9000, 10)
10


In [14]:
x_ = tf.placeholder(tf.float32,[None,num_features])
y_ = tf.placeholder(tf.float32,[None,num_labels])

In [15]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.sigmoid(layer_1)
    
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.sigmoid(layer_2)
    
    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.sigmoid(layer_3)
    
    # Hidden layer with sigmoid activation
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4 = tf.sigmoid(layer_4)
     
    # Hidden layer with sigmoid activation
    layer_5 = tf.add(tf.matmul(layer_4,weights['h5']),biases['b5'])
    layer_5 = tf.sigmoid(layer_5)
    
    # Output layer with linear activation
#     out_layer = tf.matmul(layer_5, weights['out']) + biases['out']
    return layer_5

In [16]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'h5': tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5])),
#     'out': tf.Variable(tf.random_normal([n_hidden_5, num_labels]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'b5': tf.Variable(tf.random_normal([n_hidden_5])),
#     'out': tf.Variable(tf.random_normal([num_labels]))
}

In [17]:
# Construct model
pred = multilayer_perceptron(x_, weights, biases)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, targets=y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()
# Test model using the test data generated in 1
correct_prediction = tf.equal(tf.round(pred),y_)
# correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y_,1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [ ]:
# Train the network
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        batch_sz=100
        accr = []
        total_len=np.shape(trainX)[0]
        total_batch = int(total_len/batch_sz)
        for i in xrange(0,total_len,batch_sz):
            acc,_,c = sess.run([accuracy,optimizer,cost],\
                           feed_dict={x_:trainX[i:i+batch_sz],\
                                      y_:trainY[i:i+batch_sz]})
            avg_cost += c/total_batch
            accr.append(acc)
        acc = np.mean(accr)
        if epoch % display_step == 0:
            print('Epoch: {0}, Cost: {1:.9f}, Accuracy: {2:.4f}'.format(epoch,avg_cost,acc))
    print("Done with training!!!")
# Test model using the test data generated in 1
# correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y_,1))
# # Calculate accuracy
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print("final cost on test set: %s" %str(sess.run(accuracy, feed_dict={x_:testx, y_:testy})))

Epoch: 0, Cost: 0.840892688, Accuracy: 0.4418
Epoch: 500, Cost: 0.623696958, Accuracy: 0.8072
Epoch: 1000, Cost: 0.585612322, Accuracy: 0.8636
Epoch: 1500, Cost: 0.578584262, Accuracy: 0.8752
Epoch: 2000, Cost: 0.575373309, Accuracy: 0.8794
Epoch: 2500, Cost: 0.573445803, Accuracy: 0.8830
Epoch: 3000, Cost: 0.572142512, Accuracy: 0.8851
Epoch: 3500, Cost: 0.571195187, Accuracy: 0.8868
Epoch: 4000, Cost: 0.570457386, Accuracy: 0.8880
Epoch: 4500, Cost: 0.569855185, Accuracy: 0.8887


In [ ]:
bb = trainY[i:i+batch_sz]

In [ ]:
np.shape(bb)